# Install Pytorch and PyG

In [ ]:
# Install required packages.
import os
import torch
os.environ['TORCH'] = torch.__version__

!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

# Customizing Aggregations within Message Passing

Aggregation functions play an important role in the message passing framework and the readout function when implementing GNNs. Many works in the GNN literature ([Hamilton et al. (2017)](https://cs.stanford.edu/~jure/pubs/graphsage-nips17.pdf), [Xu et al. (2018)](https://arxiv.org/abs/1810.00826), [Corso et al. (2020)](https://proceedings.neurips.cc/paper/2020/file/99cad265a1768cc2dd013f0e740300ae-Paper.pdf), [Li et al. (2020)](https://arxiv.org/abs/2006.07739)), demonstrate that the choice of aggregation functions contributes significantly to the performance of GNN models. In particular, the performance of GNNs with different aggregation functions differs when applied to distinct tasks and datasets. Recent works also show that using multiple aggregations ([Corso et al. (2020)](https://proceedings.neurips.cc/paper/2020/file/99cad265a1768cc2dd013f0e740300ae-Paper.pdf)) and learnable aggregations ([Li et al. (2020)](https://arxiv.org/abs/2006.07739)) can potentially gain substantial improvements. To facilitate experimentation with these different aggregation schemes and unify concepts of aggregation within GNNs across both [`MessagePassing`](https://github.com/pyg-team/pytorch_geometric/blob/master/torch_geometric/nn/conv/message_passing.py) and [global readouts](https://github.com/pyg-team/pytorch_geometric/tree/master/torch_geometric/nn/glob), we provide **modular and re-usable aggregations** in the newly defined `torch_geometric.nn.aggr.*` package. Unifying these concepts also helps us to perform optimization and specialized implementations in a single place. In the new integration, the following functionality is applicable:

```python
# Original interface with string type as aggregation argument
class MyConv(MessagePassing):
    def __init__(self):
        super().__init__(aggr="mean")


# Use a single aggregation module as aggregation argument
class MyConv(MessagePassing):
    def __init__(self):
        super().__init__(aggr=MeanAggregation())


# Use a list of aggregation strings as aggregation argument
class MyConv(MessagePassing):
    def __init__(self):
        super().__init__(aggr=['mean', 'max', 'sum', 'std', 'var'])


# Use a list of aggregation modules as aggregation argument
class MyConv(MessagePassing):
    def __init__(self):
        super().__init__(aggr=[
            MeanAggregation(),
            MaxAggregation(),
            SumAggregation(),
            StdAggregation(),
            VarAggregation(),
        ])


# Use a list of mixed modules and strings as aggregation argument
class MyConv(MessagePassing):
    def __init__(self):
        super().__init__(aggr=[
            'mean',
            MaxAggregation(),
            'sum',
            StdAggregation(),
            'var',
        ])


# Define multiple aggregations with `MultiAggregation` module
class MyConv(MessagePassing):
    def __init__(self):
        super().__init__(aggr=MultiAggregation([
            SoftmaxAggregation(t=0.1, learn=True),
            SoftmaxAggregation(t=1, learn=True),
            SoftmaxAggregation(t=10, learn=True)
        ]))

```



In this tutorial, we explore the new aggregation package with `SAGEConv` ([Hamilton et al. (2017)](https://cs.stanford.edu/~jure/pubs/graphsage-nips17.pdf)) and `ClusterLoader` ([Chiang et al. (2019)](https://arxiv.org/abs/1905.07953)) and showcase on the `PubMed` graph from the `Planetoid` node classification benchmark suite ([Yang et al. (2016)](https://arxiv.org/abs/1603.08861)).

## Loading the dataset
Let's first load the `Planetoid` dataset and create subgraphs with `ClusterData` for training.

In [ ]:
import torch
from torch_geometric.datasets import Planetoid
from torch_geometric.transforms import NormalizeFeatures

dataset = Planetoid(root='data/Planetoid', name='PubMed',
                    transform=NormalizeFeatures())

print()
print(f'Dataset: {dataset}:')
print('==================')
print(f'Number of graphs: {len(dataset)}')
print(f'Number of features: {dataset.num_features}')
print(f'Number of classes: {dataset.num_classes}')

data = dataset[0]  # Get the first graph object.

print()
print(data)

from torch_geometric.loader import ClusterData, ClusterLoader

seed = 42
torch.manual_seed(seed)
cluster_data = ClusterData(data, num_parts=128)  # 1. Create subgraphs.
train_loader = ClusterLoader(cluster_data, batch_size=32,
                             shuffle=True)  # 2. Stochastic partioning scheme.


Dataset: PubMed():
Number of graphs: 1
Number of features: 500
Number of classes: 3

Data(x=[19717, 500], edge_index=[2, 88648], y=[19717], train_mask=[19717], val_mask=[19717], test_mask=[19717])


Computing METIS partitioning...
Done!


## Define train, test, and run functions
Here we define a simple `run` function for training the GNN model.

In [ ]:
criterion = torch.nn.CrossEntropyLoss()


def train(model):
    model.train()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01,
                                 weight_decay=5e-4)
    for sub_data in train_loader:  # Iterate over each mini-batch.
        optimizer.zero_grad()  # Clear gradients.
        out = model(sub_data.x,
                    sub_data.edge_index)  # Perform a single forward pass.
        loss = criterion(
            out[sub_data.train_mask], sub_data.y[sub_data.train_mask]
        )  # Compute the loss solely based on the training nodes.
        loss.backward()  # Derive gradients.
        optimizer.step()  # Update parameters based on gradients.


def test(model):
    model.eval()
    out = model(data.x, data.edge_index)
    pred = out.argmax(dim=1)  # Use the class with highest probability.

    accs = []
    for mask in [data.train_mask, data.val_mask, data.test_mask]:
        correct = pred[mask] == data.y[
            mask]  # Check against ground-truth labels.
        accs.append(int(correct.sum()) /
                    int(mask.sum()))  # Derive ratio of correct predictions.
    return accs


def run(model, epochs=5):
    for epoch in range(epochs):
        loss = train(model)
        train_acc, val_acc, test_acc = test(model)
        print(
            f'Epoch: {epoch:03d}, Train: {train_acc:.4f}, Val Acc: {val_acc:.4f}, Test Acc: {test_acc:.4f}'
        )

## Define a GNN class and Import Aggregations
Now, let's define a GNN helper class and import all those new aggregation operators!




In [ ]:
import copy

import torch.nn.functional as F
from torch_geometric.nn import (
    Aggregation,
    MaxAggregation,
    MeanAggregation,
    MultiAggregation,
    SAGEConv,
    SoftmaxAggregation,
    StdAggregation,
    SumAggregation,
    VarAggregation,
)


class GNN(torch.nn.Module):
    def __init__(self, hidden_channels, aggr='mean', aggr_kwargs=None):
        super().__init__()
        self.conv1 = SAGEConv(
            dataset.num_node_features,
            hidden_channels,
            aggr=aggr,
            aggr_kwargs=aggr_kwargs,
        )
        self.conv2 = SAGEConv(
            hidden_channels,
            dataset.num_classes,
            aggr=copy.deepcopy(aggr),
            aggr_kwargs=aggr_kwargs,
        )

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv2(x, edge_index)
        return x

### Original interface with string type as the aggregation argument
Previously, PyG only supports customizing [MessagePassing](https://github.com/pyg-team/pytorch_geometric/blob/master/torch_geometric/nn/conv/message_passing.py) with simple aggregations (e.g., `'mean'`, `'max'`, `'sum'`). Let's define a GNN with `mean` aggregation and run it for 5 epochs.

In [ ]:
torch.manual_seed(seed)
model = GNN(16, aggr='mean')
print(model)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
run(model)

GNN(
  (conv1): SAGEConv(500, 16, aggr=mean)
  (conv2): SAGEConv(16, 3, aggr=mean)
)
Epoch: 000, Train: 0.6000, Val Acc: 0.4400, Test Acc: 0.4240
Epoch: 001, Train: 0.6167, Val Acc: 0.5380, Test Acc: 0.5120
Epoch: 002, Train: 0.6500, Val Acc: 0.5440, Test Acc: 0.5180
Epoch: 003, Train: 0.7167, Val Acc: 0.5540, Test Acc: 0.5250
Epoch: 004, Train: 0.9000, Val Acc: 0.7200, Test Acc: 0.6920


### Use a single aggregation module as the aggregation argument
In the new interface, the [MessagePassing](https://pytorch-geometric.readthedocs.io/en/latest/modules/nn.html#torch_geometric.nn.conv.MessagePassing) class can take an [Aggregation](https://pytorch-geometric.readthedocs.io/en/latest/modules/nn.html#torch_geometric.nn.aggr.Aggregation) module as an argument. Here we can define the mean aggregation by `MeanAggregation`. We can see the model achieves the same performance as previously.

In [ ]:
torch.manual_seed(seed)
model = GNN(16, aggr=MeanAggregation())
print(model)
run(model)

GNN(
  (conv1): SAGEConv(500, 16, aggr=MeanAggregation())
  (conv2): SAGEConv(16, 3, aggr=MeanAggregation())
)
Epoch: 000, Train: 0.6000, Val Acc: 0.4400, Test Acc: 0.4240
Epoch: 001, Train: 0.6167, Val Acc: 0.5380, Test Acc: 0.5120
Epoch: 002, Train: 0.6500, Val Acc: 0.5440, Test Acc: 0.5180
Epoch: 003, Train: 0.7167, Val Acc: 0.5540, Test Acc: 0.5250
Epoch: 004, Train: 0.9000, Val Acc: 0.7200, Test Acc: 0.6920


### Use a list of aggregation strings as the aggregation argument

For defining multiple aggregations, we can use a list of strings as the input argument. The aggregations will be **resolved from pure strings** via a lookup table, following the design principles of the [class-resolver](https://github.com/cthoyt/class-resolver) library, e.g., by simply passing in `"mean"` to the [**MessagePassing**](https://pytorch-geometric.readthedocs.io/en/latest/modules/nn.html#torch_geometric.nn.conv.MessagePassing) module. This will automatically resolve it to the MeanAggregation class. Let's see how a PNA-like GNN ([Corso et al. (2020)](https://proceedings.neurips.cc/paper/2020/file/99cad265a1768cc2dd013f0e740300ae-Paper.pdf)) works. It converges much faster!

In [ ]:
torch.manual_seed(seed)
model = GNN(16, aggr=['mean', 'max', 'sum', 'std', 'var'])
print(model)
run(model)

GNN(
  (conv1): SAGEConv(500, 16, aggr=['mean', 'max', 'sum', 'std', 'var'])
  (conv2): SAGEConv(16, 3, aggr=['mean', 'max', 'sum', 'std', 'var'])
)
Epoch: 000, Train: 0.7833, Val Acc: 0.6420, Test Acc: 0.6320
Epoch: 001, Train: 0.8333, Val Acc: 0.7040, Test Acc: 0.6980
Epoch: 002, Train: 0.8500, Val Acc: 0.6660, Test Acc: 0.6570
Epoch: 003, Train: 0.9500, Val Acc: 0.7120, Test Acc: 0.7010
Epoch: 004, Train: 0.9333, Val Acc: 0.7500, Test Acc: 0.7420


### Use a list of aggregation modules as the aggregation argument
You can also use a list of [Aggregation](https://pytorch-geometric.readthedocs.io/en/latest/modules/nn.html#torch_geometric.nn.aggr.Aggregation) modules to specify your convolutions.

In [ ]:
torch.manual_seed(seed)
model = GNN(
    16, aggr=[
        MeanAggregation(),
        MaxAggregation(),
        SumAggregation(),
        StdAggregation(),
        VarAggregation(),
    ])
print(model)
run(model)

GNN(
  (conv1): SAGEConv(500, 16, aggr=['MeanAggregation()', 'MaxAggregation()', 'SumAggregation()', 'StdAggregation()', 'VarAggregation()'])
  (conv2): SAGEConv(16, 3, aggr=['MeanAggregation()', 'MaxAggregation()', 'SumAggregation()', 'StdAggregation()', 'VarAggregation()'])
)
Epoch: 000, Train: 0.7833, Val Acc: 0.6420, Test Acc: 0.6320
Epoch: 001, Train: 0.8333, Val Acc: 0.7040, Test Acc: 0.6980
Epoch: 002, Train: 0.8500, Val Acc: 0.6660, Test Acc: 0.6570
Epoch: 003, Train: 0.9500, Val Acc: 0.7120, Test Acc: 0.7010
Epoch: 004, Train: 0.9333, Val Acc: 0.7500, Test Acc: 0.7420


### Use a list of mixed modules and strings as the aggregation argument
And the mix of them is supported as well for your convenience.

In [ ]:
torch.manual_seed(seed)
model = GNN(16, aggr=[
    'mean',
    MaxAggregation(),
    'sum',
    StdAggregation(),
    'var',
])
print(model)
run(model)

GNN(
  (conv1): SAGEConv(500, 16, aggr=['mean', 'MaxAggregation()', 'sum', 'StdAggregation()', 'var'])
  (conv2): SAGEConv(16, 3, aggr=['mean', 'MaxAggregation()', 'sum', 'StdAggregation()', 'var'])
)
Epoch: 000, Train: 0.7833, Val Acc: 0.6420, Test Acc: 0.6320
Epoch: 001, Train: 0.8333, Val Acc: 0.7040, Test Acc: 0.6980
Epoch: 002, Train: 0.8500, Val Acc: 0.6660, Test Acc: 0.6570
Epoch: 003, Train: 0.9500, Val Acc: 0.7120, Test Acc: 0.7010
Epoch: 004, Train: 0.9333, Val Acc: 0.7500, Test Acc: 0.7420


### Define multiple aggregations with `MultiAggregation` module

When a list is taken, [MessagePassing](https://pytorch-geometric.readthedocs.io/en/latest/modules/nn.html#torch_geometric.nn.conv.MessagePassing) would stack these aggregators in via the [MultiAggregation](https://pytorch-geometric.readthedocs.io/en/latest/modules/nn.html#torch_geometric.nn.aggr.MultiAggregation) module automatically. But you can also directly pass a [MultiAggregation](https://pytorch-geometric.readthedocs.io/en/latest/modules/nn.html#torch_geometric.nn.aggr.MultiAggregation) instead of a list. Now let's see how can we define multiple aggregations with [MultiAggregation](https://pytorch-geometric.readthedocs.io/en/latest/modules/nn.html#torch_geometric.nn.aggr.MultiAggregation). Here we use different initial temperatures for [SoftmaxAggregation](https://pytorch-geometric.readthedocs.io/en/latest/modules/nn.html#torch_geometric.nn.aggr.SoftmaxAggregation) ([Li et al. (2020)](https://arxiv.org/abs/2006.07739)). Every different temperature will result in aggregation with different softness.

In [ ]:
torch.manual_seed(seed)
aggr = MultiAggregation([
    SoftmaxAggregation(t=0.01, learn=True),
    SoftmaxAggregation(t=1, learn=True),
    SoftmaxAggregation(t=100, learn=True),
])
model = GNN(16, aggr=aggr)
print(model)
run(model)

GNN(
  (conv1): SAGEConv(500, 16, aggr=MultiAggregation([
    SoftmaxAggregation(learn=True),
    SoftmaxAggregation(learn=True),
    SoftmaxAggregation(learn=True)
  ], mode=cat))
  (conv2): SAGEConv(16, 3, aggr=MultiAggregation([
    SoftmaxAggregation(learn=True),
    SoftmaxAggregation(learn=True),
    SoftmaxAggregation(learn=True)
  ], mode=cat))
)
Epoch: 000, Train: 0.7333, Val Acc: 0.6080, Test Acc: 0.6340
Epoch: 001, Train: 0.8500, Val Acc: 0.6440, Test Acc: 0.6560
Epoch: 002, Train: 0.9333, Val Acc: 0.7420, Test Acc: 0.7390
Epoch: 003, Train: 0.9333, Val Acc: 0.7560, Test Acc: 0.7360
Epoch: 004, Train: 0.9833, Val Acc: 0.7840, Test Acc: 0.7460


What is more?
There are many other aggregation operators supported for you to "lego" your GNNs. [PowerMeanAggregation](https://pytorch-geometric.readthedocs.io/en/latest/modules/nn.html#torch_geometric.nn.aggr.PowerMeanAggregation) allows you to define and potentially learn generalized means beyond simple  arithmetic mean such as harmonic mean and geometric mean. [LSTMAggregation](https://pytorch-geometric.readthedocs.io/en/latest/modules/nn.html#torch_geometric.nn.aggr.LSTMAggregation) can perform permutation-variant aggregation. More other interesting aggregation operators such as [Set2Set](https://pytorch-geometric.readthedocs.io/en/latest/modules/nn.html#torch_geometric.nn.aggr.Set2Set), [DegreeScalerAggregation](https://pytorch-geometric.readthedocs.io/en/latest/modules/nn.html#torch_geometric.nn.aggr.DegreeScalerAggregation), [SortAggregation](https://pytorch-geometric.readthedocs.io/en/latest/modules/nn.html#torch_geometric.nn.aggr.SortAggregation), [GraphMultisetTransformer](https://pytorch-geometric.readthedocs.io/en/latest/modules/nn.html#torch_geometric.nn.aggr.GraphMultisetTransformer), [AttentionalAggregation](https://pytorch-geometric.readthedocs.io/en/latest/modules/nn.html#torch_geometric.nn.aggr.AttentionalAggregation) and [EquilibriumAggregation](https://pytorch-geometric.readthedocs.io/en/latest/modules/nn.html#torch_geometric.nn.aggr.EquilibriumAggregation) are ready for you to explore.

## Conclusion

In this tutorial, you have been presented with the `torch_geometric.nn.aggr` package which provides a flexible interface to experiment with different aggregation functions with your message passing convolutions and unifies aggregation within GNNs across [`MessagePassing`](https://github.com/pyg-team/pytorch_geometric/blob/master/torch_geometric/nn/conv/message_passing.py) and [global readouts](https://github.com/pyg-team/pytorch_geometric/tree/master/torch_geometric/nn/glob). This new abstraction also makes designing new types of aggregation functions easier. Now, you can create your own aggregation function with the base `Aggregation` class. Please refer to the [docs](https://pytorch-geometric.readthedocs.io/en/latest/_modules/torch_geometric/nn/aggr/base.html#Aggregation) for more details.

```python
class MyAggregation(Aggregation):
    def __init__(self, ...):
      ...

    def forward(self, x: Tensor, index: Optional[Tensor] = None,
                ptr: Optional[Tensor] = None, dim_size: Optional[int] = None,
                dim: int = -2) -> Tensor:
      ...
```

*Have fun!*